In [175]:
import pandas as pd

In [176]:
# clean benchmark log
import csv

with open('benchmark_log.csv', 'r') as fin, open('benchmark_log_cleaned.csv', 'w') as fout:
    data = [row[:9] for row in csv.reader(fin)]
    csv.writer(fout).writerows(data)

In [177]:
data = pd.read_csv('benchmark_log_cleaned.csv')

# remove unnecessary columns
data.drop("train_steps", axis=1, inplace=True)

# get total cost in scientific notation
data.drop("total_cost_format", axis=1, inplace=True)
data["total_cost_format"] = data["total_cost"].apply(lambda x: f"{x:.4e}")

# convert microgrid to IDs
unique_microgrids = data["microgrid"].unique()
microgrid_map = {name: i for i, name in enumerate(unique_microgrids)}
data["microgrid"] = data["microgrid"].map(microgrid_map)

data.head()

,config_id,total_cost,microgrid,agent,policy_act,policy_net_arch,learning_rate,total_cost_format
0,0,53062.168702,0,SB3Agent DQN,ReLU,"[64, 64]",0.0001,5.3062e+04
1,1,55578.578584,0,SB3Agent DQN,ReLU,"[64, 64]",0.0005,5.5579e+04
2,2,57543.274309,0,SB3Agent DQN,ReLU,"[64, 64]",0.0010,5.7543e+04
3,3,55934.748791,0,SB3Agent DQN,ReLU,"[128, 128]",0.0001,5.5935e+04
4,4,84487.924273,0,SB3Agent DQN,ReLU,"[128, 128]",0.0005,8.4488e+04


In [178]:
heuristic_res = [
    item["total_cost"] for item in data[data["agent"] == "BasicAgent heuristics"]\
    .drop_duplicates(subset=["microgrid"])[["total_cost"]]\
    .to_dict(orient="records")
]

data = data[data["agent"] != "BasicAgent heuristics"].copy()

def add_baseline(row):
    return heuristic_res[row["microgrid"]]

def absolute_improvement(row):
    return heuristic_res[row["microgrid"]] - row["total_cost"]

def add_percentage_improvement(row):
    heuristic_cost = heuristic_res[row["microgrid"]]
    return (heuristic_cost - row["total_cost"]) / heuristic_cost * 100

data["baseline"] = data.apply(add_baseline, axis=1)
data["absolute_improvement"] = data.apply(absolute_improvement, axis=1)
data["percentage_improvement"] = data.apply(add_percentage_improvement, axis=1)

data.head()

,config_id,total_cost,microgrid,agent,policy_act,policy_net_arch,learning_rate,total_cost_format,baseline,absolute_improvement,percentage_improvement
0,0,53062.168702,0,SB3Agent DQN,ReLU,"[64, 64]",0.0001,5.3062e+04,187183.490996,134121.322294,71.652324
1,1,55578.578584,0,SB3Agent DQN,ReLU,"[64, 64]",0.0005,5.5579e+04,187183.490996,131604.912412,70.307970
2,2,57543.274309,0,SB3Agent DQN,ReLU,"[64, 64]",0.0010,5.7543e+04,187183.490996,129640.216687,69.258360
3,3,55934.748791,0,SB3Agent DQN,ReLU,"[128, 128]",0.0001,5.5935e+04,187183.490996,131248.742206,70.117691
4,4,84487.924273,0,SB3Agent DQN,ReLU,"[128, 128]",0.0005,8.4488e+04,187183.490996,102695.566723,54.863581


In [179]:
[f"{res:.4e}" for res in heuristic_res]

['1.8718e+05',
 '2.3694e+07',
 '3.6006e+07',
 '3.5932e+07',
 '3.2445e+07',
 '1.1351e+08',
 '4.6285e+07',
 '1.5447e+07',
 '9.7355e+06',
 '3.5711e+06']

In [180]:
# get best general configs according to total_cost
data.sort_values("total_cost").head()

,config_id,total_cost,microgrid,agent,policy_act,policy_net_arch,learning_rate,total_cost_format,baseline,absolute_improvement,percentage_improvement
34,34,51143.508873,0,SB3Agent PPO,Tanh,"[128, 128]",0.0005,5.1144e+04,187183.490996,136039.982124,72.67734
18,18,51143.508873,0,SB3Agent A2C,Tanh,"[64, 64]",0.0001,5.1144e+04,187183.490996,136039.982124,72.67734
17,17,51143.508873,0,SB3Agent A2C,ReLU,"[128, 128]",0.0010,5.1144e+04,187183.490996,136039.982124,72.67734
21,21,51143.508873,0,SB3Agent A2C,Tanh,"[128, 128]",0.0001,5.1144e+04,187183.490996,136039.982124,72.67734
15,15,51143.508873,0,SB3Agent A2C,ReLU,"[128, 128]",0.0001,5.1144e+04,187183.490996,136039.982124,72.67734


In [181]:
# get best general configs according to total_cost
data.sort_values("percentage_improvement", ascending=False).head()

,config_id,total_cost,microgrid,agent,policy_act,policy_net_arch,learning_rate,total_cost_format,baseline,absolute_improvement,percentage_improvement
202,202,4.073472e+06,5,SB3Agent DQN,Tanh,"[64, 64]",0.0005,4.0735e+06,1.135082e+08,1.094347e+08,96.411296
230,230,4.073472e+06,5,SB3Agent PPO,Tanh,"[128, 128]",0.0010,4.0735e+06,1.135082e+08,1.094347e+08,96.411296
229,229,4.073472e+06,5,SB3Agent PPO,Tanh,"[128, 128]",0.0005,4.0735e+06,1.135082e+08,1.094347e+08,96.411296
227,227,4.073472e+06,5,SB3Agent PPO,Tanh,"[64, 64]",0.0010,4.0735e+06,1.135082e+08,1.094347e+08,96.411296
226,226,4.073472e+06,5,SB3Agent PPO,Tanh,"[64, 64]",0.0005,4.0735e+06,1.135082e+08,1.094347e+08,96.411296


In [165]:
# for each microgrid, get the best config
for microgrid in range(len(unique_microgrids)):
    best_config = data[data["microgrid"] == microgrid].sort_values("total_cost").head(1)
    print(f"Best config for microgrid {microgrid}:")
    print(best_config.to_string(index=False))
    print()

Best config for microgrid 0:
 config_id   total_cost  microgrid        agent policy_act policy_net_arch  learning_rate total_cost_format      baseline  percentage_improvement
        17 51143.508873          0 SB3Agent A2C       ReLU      [128, 128]          0.001        5.1144e+04 187183.490996                72.67734

Best config for microgrid 1:
 config_id   total_cost  microgrid        agent policy_act policy_net_arch  learning_rate total_cost_format     baseline  percentage_improvement
        39 3.646774e+06          1 SB3Agent DQN       ReLU        [64, 64]         0.0001        3.6468e+06 2.369357e+07               84.608591

Best config for microgrid 2:
 config_id   total_cost  microgrid        agent policy_act policy_net_arch  learning_rate total_cost_format     baseline  percentage_improvement
        78 1.750680e+06          2 SB3Agent DQN       ReLU        [64, 64]         0.0001        1.7507e+06 3.600570e+07               95.137769

Best config for microgrid 3:
 config_i

In [166]:
data.drop(columns=["total_cost_format", "microgrid", "config_id", "baseline", "absolute_improvement", "percentage_improvement"])\
    .groupby(["agent", "policy_act", "policy_net_arch", "learning_rate"])\
    .mean()\
    .sort_values("total_cost")

total_cost
agent        policy_act policy_net_arch learning_rate              
SB3Agent PPO Tanh       [64, 64]        0.0010         2.403480e+06
                        [128, 128]      0.0010         2.403480e+06
                                        0.0005         2.403480e+06
                        [64, 64]        0.0005         2.403480e+06
SB3Agent A2C Tanh       [128, 128]      0.0010         2.403480e+06
SB3Agent PPO Tanh       [64, 64]        0.0001         2.403504e+06
                        [128, 128]      0.0001         2.403504e+06
SB3Agent A2C Tanh       [64, 64]        0.0010         2.403517e+06
                        [128, 128]      0.0005         2.403517e+06
SB3Agent DQN Tanh       [128, 128]      0.0001         2.403525e+06
SB3Agent A2C Tanh       [128, 128]      0.0001         2.403624e+06
                        [64, 64]        0.0001         2.403624e+06
                                        0.0005         2.403624e+06
SB3Agent DQN Tanh       [64, 64]        0.0005         2.406824e+06
             ReLU       [64, 64]        0.0001         2.425341e+06
                        [128, 128]      0.0001         2.447507e+06
             Tanh       [64, 64]        0.0010         2.457281e+06
                        [128, 128]      0.0010         2.466014e+06
                        [64, 64]        0.0001         2.619356e+06
                        [128, 128]      0.0005         2.653883e+06
             ReLU       [128, 128]      0.0005         2.693779e+06
                        [64, 64]        0.0005         2.742951e+06
                        [128, 128]      0.0010         3.192899e+06
                        [64, 64]        0.0010         3.815355e+06
SB3Agent PPO ReLU       [128, 128]      0.0001         5.870733e+06
SB3Agent A2C ReLU       [64, 64]        0.0001         7.446808e+06
SB3Agent PPO ReLU       [64, 64]        0.0005         9.505715e+06
                        [128, 128]      0.0010         1.474182e+07
                        [64, 64]        0.0001         1.707237e+07
                                        0.0010         1.711980e+07
                        [128, 128]      0.0005         1.012782e+08
SB3Agent A2C ReLU       [64, 64]        0.0010         2.468530e+08
                                        0.0005         2.665263e+08
                        [128, 128]      0.0001         2.777333e+08
                                        0.0010         3.050325e+08
                                        0.0005         3.297995e+08

In [167]:
data.drop(columns=["total_cost_format", "microgrid", "config_id", "baseline", "absolute_improvement", "percentage_improvement", "learning_rate"])\
    .groupby(["agent", "policy_act", "policy_net_arch"])\
    .mean()\
    .sort_values("total_cost")

total_cost
agent        policy_act policy_net_arch              
SB3Agent PPO Tanh       [128, 128]       2.403488e+06
                        [64, 64]         2.403488e+06
SB3Agent A2C Tanh       [128, 128]       2.403540e+06
                        [64, 64]         2.403588e+06
SB3Agent DQN Tanh       [64, 64]         2.494487e+06
                        [128, 128]       2.507807e+06
             ReLU       [128, 128]       2.778062e+06
                        [64, 64]         2.994549e+06
SB3Agent PPO ReLU       [64, 64]         1.456596e+07
                        [128, 128]       4.063024e+07
SB3Agent A2C ReLU       [64, 64]         1.736087e+08
                        [128, 128]       3.041885e+08

In [188]:
res = data.drop(columns=["total_cost_format", "microgrid", "config_id", "baseline", "total_cost", "absolute_improvement"])\
    .groupby(["agent", "policy_act", "policy_net_arch", "learning_rate"])\
    .agg(
        mean_percentage_improvement=("percentage_improvement", "mean"),
        std_percentage_improvement=("percentage_improvement", "std"),
    )\
    .sort_values("mean_percentage_improvement", ascending=False)

res[res["mean_percentage_improvement"] > 0]

mean_percentage_improvement  \
agent        policy_act policy_net_arch learning_rate                                
SB3Agent PPO Tanh       [64, 64]        0.0010                           82.753920   
                                        0.0005                           82.753920   
                        [128, 128]      0.0010                           82.753920   
                                        0.0005                           82.753920   
SB3Agent A2C Tanh       [128, 128]      0.0010                           82.753920   
SB3Agent PPO Tanh       [128, 128]      0.0001                           82.753899   
                        [64, 64]        0.0001                           82.753899   
SB3Agent A2C Tanh       [128, 128]      0.0005                           82.753812   
                        [64, 64]        0.0010                           82.753812   
                        [128, 128]      0.0001                           82.753515   
                        [64, 64]        0.0001                           82.753515   
                                        0.0005                           82.753515   
SB3Agent DQN Tanh       [128, 128]      0.0001                           82.729874   
             ReLU       [64, 64]        0.0001                           82.608700   
             Tanh       [64, 64]        0.0005                           82.526262   
                                        0.0010                           82.426822   
                        [128, 128]      0.0010                           82.426435   
             ReLU       [128, 128]      0.0001                           82.363391   
             Tanh       [64, 64]        0.0001                           82.287536   
                        [128, 128]      0.0005                           81.767518   
             ReLU       [64, 64]        0.0005                           80.551613   
                        [128, 128]      0.0005                           80.358122   
                                        0.0010                           79.025485   
SB3Agent A2C ReLU       [64, 64]        0.0001                           78.301247   
SB3Agent DQN ReLU       [64, 64]        0.0010                           77.783150   
SB3Agent PPO ReLU       [64, 64]        0.0005                           73.857371   
                        [128, 128]      0.0001                           73.104358   
                        [64, 64]        0.0001                           39.511530   
                                        0.0010                           38.792167   

                                                       std_percentage_improvement  
agent        policy_act policy_net_arch learning_rate                              
SB3Agent PPO Tanh       [64, 64]        0.0010                          17.239815  
                                        0.0005                          17.239815  
                        [128, 128]      0.0010                          17.239815  
                                        0.0005                          17.239815  
SB3Agent A2C Tanh       [128, 128]      0.0010                          17.239815  
SB3Agent PPO Tanh       [128, 128]      0.0001                          17.239797  
                        [64, 64]        0.0001                          17.239797  
SB3Agent A2C Tanh       [128, 128]      0.0005                          17.239745  
                        [64, 64]        0.0010                          17.239745  
                        [128, 128]      0.0001                          17.239499  
                        [64, 64]        0.0001                          17.239499  
                                        0.0005                          17.239499  
SB3Agent DQN Tanh       [128, 128]      0.0001                          17.255592  
             ReLU       [64, 64]        0.0001                          17.284568  
             Tanh       [64, 64]        0.00

In [189]:
res = data.drop(columns=["total_cost_format", "microgrid", "config_id", "baseline", "total_cost", "absolute_improvement", "learning_rate"])\
    .groupby(["agent", "policy_act", "policy_net_arch"])\
    .agg(
        mean_percentage_improvement=("percentage_improvement", "mean"),
        std_percentage_improvement=("percentage_improvement", "std"),
    )\
    .sort_values("mean_percentage_improvement", ascending=False)

res[res["mean_percentage_improvement"] > 0]

mean_percentage_improvement  \
agent        policy_act policy_net_arch                                
SB3Agent PPO Tanh       [128, 128]                         82.753913   
                        [64, 64]                           82.753913   
SB3Agent A2C Tanh       [128, 128]                         82.753749   
                        [64, 64]                           82.753614   
SB3Agent DQN Tanh       [64, 64]                           82.413540   
                        [128, 128]                         82.307943   
             ReLU       [128, 128]                         80.582333   
                        [64, 64]                           80.314488   
SB3Agent PPO ReLU       [64, 64]                           50.720356   

                                         std_percentage_improvement  
agent        policy_act policy_net_arch                              
SB3Agent PPO Tanh       [128, 128]                        16.634714  
                        [64, 64]                          16.634714  
SB3Agent A2C Tanh       [128, 128]                        16.634595  
                        [64, 64]                          16.634493  
SB3Agent DQN Tanh       [64, 64]                          16.617307  
                        [128, 128]                        16.787641  
             ReLU       [128, 128]                        17.538421  
                        [64, 64]                          18.051209  
SB3Agent PPO ReLU       [64, 64]                          49.075062